# Sample Workflow for d3rlpy Experiments

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import math
import subprocess
import os
import d3rlpy
plt.style.use('matplotlibrc')

from Python.data_sampler import *

## Building an MDPDataset

We first read in a large batch of samples from the file. As `d3rlpy` wants it in the form (observations, actions, rewards, terminal flags), we go ahead and do that. Here's a helper function to get a dataset from a list of chunks of your choosing.

In [2]:
def get_dataset(chunks : list, batch_size=30000, 
                path="collected_data/rl_det_small.txt") -> d3rlpy.dataset.MDPDataset :
    random.seed(0)
    samples = DataSampler(path_to_data=path)
    states = []
    actions = []
    rewards = []
    next_states = []
    for chunk in chunks:
        samples.use_chunk(chunk)
        samples.read_chunk()
        [statesChunk, actionsChunk, rewardsChunk, nextStatesChunk] = samples.get_batch(batch_size)
        states.append(statesChunk)
        actions.append(actionsChunk)
        rewards.append(rewardsChunk)
        next_states.append(nextStatesChunk)
    states = torch.cat(states)
    actions = torch.cat(actions)
    rewards = torch.cat(rewards)
    next_states = torch.cat(next_states)
    terminals = np.zeros(len(states))
    terminals[::100] = 1 #episode length 100, change if necessary
    print(states.shape)
    dataset = d3rlpy.dataset.MDPDataset(states.numpy(), 
                                        actions.numpy(), 
                                        rewards.numpy(), terminals)
    return dataset

We can build the dataset from there, just like this, and split into train and test sets.

In [14]:
dataset = get_dataset([3,5,7,9], path="collected_data/rl_stoch_small.txt")

[ 0.00000000e+00  7.95731469e+08 -1.23489108e-01 -3.79999531e-03
  2.19998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  7.60168026e-02 -3.40510126e-01  6.00000000e-01]
Read chunk # 4 out of 10000
[ 0.00000000e+00  7.95731469e+08 -1.28589108e-01  1.20000047e-02
  1.99998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  7.75212759e-03 -3.52719043e-01  6.00000000e-01]
Read chunk # 6 out of 10000
[ 0.00000000e+00  7.95731469e+08  9.97108923e-02 -1.79999531e-03
 -6.70001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -1.78416413e-01  3.07580560e-01 -6.00000000e-01]
Read chunk # 8 out of 10000
[ 0.00000000e+00  7.95731469e+08 -1.03989108e-01  7.00000469e-03
 -8.60001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.65974295e-01 -2.19295880e-01  6.00000000e-01]
Read chunk # 10 out of 10000
torch.Size([111080, 6])


In [15]:
print("The behavior policy value statistics are:")
dataset.compute_stats()['return']

The behavior policy value statistics are:


{'mean': -5.6349497,
 'std': 2.8907404,
 'min': -12.561853,
 'max': 0.0,
 'histogram': (array([ 4, 15, 52, 42, 32, 38, 36, 85, 65, 56, 87, 98, 90, 78, 67, 80, 86,
         54, 36, 10]),
  array([-12.561853  , -11.933761  , -11.305668  , -10.677575  ,
         -10.049482  ,  -9.421391  ,  -8.793298  ,  -8.165205  ,
          -7.537112  ,  -6.9090195 ,  -6.2809267 ,  -5.652834  ,
          -5.024741  ,  -4.396649  ,  -3.768556  ,  -3.1404634 ,
          -2.5123706 ,  -1.884278  ,  -1.2561853 ,  -0.62809265,
           0.        ], dtype=float32))}

In [16]:
from sklearn.model_selection import train_test_split
train_episodes, test_episodes = train_test_split(dataset, test_size=0.2)

## Setting up an Algorithm

In [17]:
from d3rlpy.algos import CQL

from d3rlpy.preprocessing import MinMaxActionScaler
action_scaler = MinMaxActionScaler(minimum=-0.6, maximum=0.6)
#cql = CQL(action_scaler=action_scaler)

model = CQL(q_func_factory='mean', #qr -> quantile regression q function, but you don't have to use this
            reward_scaler='standard',
            action_scaler=action_scaler,
          actor_learning_rate=1e-5, 
          critic_learning_rate=0.0003, 
            use_gpu=False) #change it to true if you have one
model.build_with_dataset(dataset)

In [18]:
from d3rlpy.metrics.scorer import td_error_scorer
from d3rlpy.metrics.scorer import average_value_estimation_scorer
from d3rlpy.metrics.scorer import initial_state_value_estimation_scorer

# calculate metrics with test dataset
ave_error_init = average_value_estimation_scorer(model, test_episodes)
print(ave_error_init)

0.14527341764378593


In [8]:
%load_ext tensorboard
%tensorboard --logdir runs

In [19]:
model.fit(train_episodes,
        eval_episodes=test_episodes,
        n_epochs=5, 
        tensorboard_dir='runs',
        scorers={
            'td_error': td_error_scorer,
            'init_value': initial_state_value_estimation_scorer,
            'ave_value': average_value_estimation_scorer
        })

2022-04-07 20:17.42 [debug    ] RoundIterator is selected.
2022-04-07 20:17.42 [info     ] Directory is created at d3rlpy_logs/CQL_20220407201742
2022-04-07 20:17.42 [debug    ] Fitting action scaler...       action_scaler=min_max
2022-04-07 20:17.42 [debug    ] Fitting reward scaler...       reward_scaler=standard
2022-04-07 20:17.42 [warning  ] Skip building models since they're already built.
2022-04-07 20:17.42 [info     ] Parameters are saved to d3rlpy_logs/CQL_20220407201742/params.json params={'action_scaler': {'type': 'min_max', 'params': {'minimum': array(-0.6), 'maximum': array(0.6)}}, 'actor_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'actor_learning_rate': 1e-05, 'actor_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'alpha_learning_rate': 0.0001, 'alpha_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight

Epoch 1/5:   0%|          | 0/343 [00:00<?, ?it/s]

2022-04-07 20:18.08 [info     ] CQL_20220407201742: epoch=1 step=343 epoch=1 metrics={'time_sample_batch': 0.0003316763886209827, 'time_algorithm_update': 0.0751432427164417, 'temp_loss': 4.685994176753408, 'temp': 0.983123943339979, 'alpha_loss': -9.663411787578038, 'alpha': 1.0142724656502637, 'critic_loss': 10.895247617198844, 'actor_loss': 0.31483166424269754, 'time_step': 0.07555461138399976, 'td_error': 3.2080431202054194, 'init_value': -0.8907099962234497, 'ave_value': -0.8926067945390094} step=343
2022-04-07 20:18.08 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20220407201742/model_343.pt


Epoch 2/5:   0%|          | 0/343 [00:00<?, ?it/s]

2022-04-07 20:18.32 [info     ] CQL_20220407201742: epoch=2 step=686 epoch=2 metrics={'time_sample_batch': 0.0003129448904587993, 'time_algorithm_update': 0.06703745310925187, 'temp_loss': 2.988745463485273, 'temp': 0.9548389737877137, 'alpha_loss': 3.04989481067866, 'alpha': 1.0203838549967061, 'critic_loss': -1.8484127205245349, 'actor_loss': 3.9301308563777377, 'time_step': 0.06742987479829927, 'td_error': 6.290793203723951, 'init_value': -2.344316005706787, 'ave_value': -2.3325591981343483} step=686
2022-04-07 20:18.32 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20220407201742/model_686.pt


...

Epoch 4/5:   0%|          | 0/343 [00:00<?, ?it/s]

2022-04-07 20:19.22 [info     ] CQL_20220407201742: epoch=4 step=1372 epoch=4 metrics={'time_sample_batch': 0.00033195859836767435, 'time_algorithm_update': 0.07073451617716353, 'temp_loss': 0.7756605448945271, 'temp': 0.9236333154728392, 'alpha_loss': 11.25295761375316, 'alpha': 0.9514596969671222, 'critic_loss': -8.302424507307936, 'actor_loss': 8.350151242737173, 'time_step': 0.07114918864503199, 'td_error': 4.421148879792159, 'init_value': -3.4863736629486084, 'ave_value': -3.456131781871948} step=1372
2022-04-07 20:19.22 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20220407201742/model_1372.pt


Epoch 5/5:   0%|          | 0/343 [00:00<?, ?it/s]

2022-04-07 20:19.50 [info     ] CQL_20220407201742: epoch=5 step=1715 epoch=5 metrics={'time_sample_batch': 0.000346282480757021, 'time_algorithm_update': 0.07982820116048651, 'temp_loss': 0.3819195845492901, 'temp': 0.9162918164153141, 'alpha_loss': 12.809436689660432, 'alpha': 0.9108857491621123, 'critic_loss': -8.761230831591103, 'actor_loss': 10.82426604724139, 'time_step': 0.08026000014547009, 'td_error': 6.248317634214251, 'init_value': -5.737633228302002, 'ave_value': -5.695264149381874} step=1715
2022-04-07 20:19.50 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20220407201742/model_1715.pt


[(1,
  {'time_sample_batch': 0.0003316763886209827,
   'time_algorithm_update': 0.0751432427164417,
   'temp_loss': 4.685994176753408,
   'temp': 0.983123943339979,
   'alpha_loss': -9.663411787578038,
   'alpha': 1.0142724656502637,
   'critic_loss': 10.895247617198844,
   'actor_loss': 0.31483166424269754,
   'time_step': 0.07555461138399976,
   'td_error': 3.2080431202054194,
   'init_value': -0.8907099962234497,
   'ave_value': -0.8926067945390094}),
 (2,
  {'time_sample_batch': 0.0003129448904587993,
   'time_algorithm_update': 0.06703745310925187,
   'temp_loss': 2.988745463485273,
   'temp': 0.9548389737877137,
   'alpha_loss': 3.04989481067866,
   'alpha': 1.0203838549967061,
   'critic_loss': -1.8484127205245349,
   'actor_loss': 3.9301308563777377,
   'time_step': 0.06742987479829927,
   'td_error': 6.290793203723951,
   'init_value': -2.344316005706787,
   'ave_value': -2.3325591981343483}),
 (3,
  {'time_sample_batch': 0.0003350066026515238,
   'time_algorithm_update': 0.07

## Off-Policy Evaluation

We do get some metrics on a test set of initial state value and average value. However, these estimates (using the critic's Q-function) of model performance are biased. They're useful for validation during training, but not much else. Instead, we fit a Q-function to the data (or a separate dataset, as I've done here) separately and evaluate the model's performance on it.

Feel free to change the chunks and number of steps.

In [20]:
from d3rlpy.ope import FQE
# metrics to evaluate with
from d3rlpy.metrics.scorer import soft_opc_scorer


ope_dataset = get_dataset([2,4,6,8], path="collected_data/rl_det_small.txt") #change if you'd prefer different chunks
ope_train_episodes, ope_test_episodes = train_test_split(ope_dataset, test_size=0.2)

fqe = FQE(algo=model, action_scaler = action_scaler, use_gpu=False) #change this if you have one!
fqe.fit(ope_train_episodes, eval_episodes=ope_test_episodes,
        tensorboard_dir='runs',
        n_epochs=50, n_steps_per_epoch=1000, #change if overfitting/underfitting
        scorers={
           'init_value': initial_state_value_estimation_scorer,
            'ave_value': average_value_estimation_scorer,
           'soft_opc': soft_opc_scorer(return_threshold=0)
        })

[ 0.00000000e+00  7.95731469e+08 -1.03891077e-02 -1.41999953e-02
 -2.10001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -1.78778459e-03 -1.34615461e-02  4.84073546e-02]
Read chunk # 3 out of 10000
[ 0.00000000e+00  7.95731469e+08 -7.24891077e-02 -1.35999953e-02
 -4.20001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.23311010e-02 -1.64283998e-01  6.00000000e-01]
Read chunk # 5 out of 10000
[ 0.00000000e+00  7.95731469e+08  7.01089229e-03 -4.19999531e-03
  7.39998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  2.21623335e-01 -2.86362315e-02 -8.00043364e-02]
Read chunk # 7 out of 10000
[ 0.00000000e+00  7.95731469e+08 -1.03989108e-01 -1.37999953e-02
  7.99998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  2.76352555e-01 -3.26280816e-01  6.00000000e-01]
Read chunk # 9 out of 10000
torch.Size([111080, 6])
2022-04-07 20:19.51 [debug    ] RoundIterator is selected.
2022-04-07 20:19.51 [info     ] Directory is created at d3rlpy_logs/FQE_2022040720195

Epoch 1/50:   0%|          | 0/878 [00:00<?, ?it/s]

2022-04-07 20:19.53 [info     ] FQE_20220407201951: epoch=1 step=878 epoch=1 metrics={'time_sample_batch': 0.00013968911963877753, 'time_algorithm_update': 0.0018511578814043813, 'loss': 0.00017434491303991263, 'time_step': 0.0020544406764871166, 'init_value': -0.14161010086536407, 'ave_value': -0.14164950612826305, 'soft_opc': nan} step=878


2022-04-07 20:19.53 [info     ] Model parameters are saved to d3rlpy_logs/FQE_20220407201951/model_878.pt


Epoch 2/50:   0%|          | 0/878 [00:00<?, ?it/s]

...

2022-04-07 20:21.40 [info     ] Model parameters are saved to d3rlpy_logs/FQE_20220407201951/model_43022.pt


Epoch 50/50:   0%|          | 0/878 [00:00<?, ?it/s]

2022-04-07 20:21.42 [info     ] FQE_20220407201951: epoch=50 step=43900 epoch=50 metrics={'time_sample_batch': 0.00014518903979950993, 'time_algorithm_update': 0.0020446207094301123, 'loss': 0.022649357612666426, 'time_step': 0.0022551255780091863, 'init_value': -1.5099941492080688, 'ave_value': -1.5100681873347797, 'soft_opc': nan} step=43900


2022-04-07 20:21.42 [info     ] Model parameters are saved to d3rlpy_logs/FQE_20220407201951/model_43900.pt


[(1,
  {'time_sample_batch': 0.00013968911963877753,
   'time_algorithm_update': 0.0018511578814043813,
   'loss': 0.00017434491303991263,
   'time_step': 0.0020544406764871166,
   'init_value': -0.14161010086536407,
   'ave_value': -0.14164950612826305,
   'soft_opc': nan}),
 (2,
  {'time_sample_batch': 0.00013360727349283482,
   'time_algorithm_update': 0.001868682461610416,
   'loss': 0.0005978871645425246,
   'time_step': 0.0020631492002135257,
   'init_value': -0.3587397336959839,
   'ave_value': -0.3586608218960169,
   'soft_opc': nan}),
 (3,
  {'time_sample_batch': 0.00013338270382892025,
   'time_algorithm_update': 0.001851950799685676,
   'loss': 0.001988239196115851,
   'time_step': 0.0020455303930747484,
   'init_value': -0.5455045104026794,
   'ave_value': -0.5452594495059389,
   'soft_opc': nan}),
 (4,
  {'time_sample_batch': 0.00013503995734629706,
   'time_algorithm_update': 0.0019145256294476416,
   'loss': 0.0036624751454680227,
   'time_step': 0.0021117569916883744,
 

In [21]:
from d3rlpy.ope import FQE
# metrics to evaluate with
from d3rlpy.metrics.scorer import soft_opc_scorer


ope_dataset = get_dataset([2,4,6,8], path="collected_data/rl_stoch_small.txt") #change if you'd prefer different chunks
ope_train_episodes, ope_test_episodes = train_test_split(ope_dataset, test_size=0.2)

fqe = FQE(algo=model, action_scaler = action_scaler, use_gpu=False) #change this if you have one!
fqe.fit(ope_train_episodes, eval_episodes=ope_test_episodes,
        tensorboard_dir='runs',
        n_epochs=50, n_steps_per_epoch=1000, #change if overfitting/underfitting
        scorers={
           'init_value': initial_state_value_estimation_scorer,
            'ave_value': average_value_estimation_scorer,
           'soft_opc': soft_opc_scorer(return_threshold=0)
        })

[ 0.00000000e+00  7.95731469e+08  1.39310892e-01  1.82000047e-02
 -1.00013420e-04  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -7.50230117e-02  3.69851546e-01 -6.00000000e-01]
Read chunk # 3 out of 10000
[ 0.00000000e+00  7.95731469e+08 -1.15389108e-01  1.64000047e-02
 -8.80001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -3.08831172e-01 -2.48178665e-01  6.00000000e-01]
Read chunk # 5 out of 10000
[ 0.00000000e+00  7.95731469e+08 -1.28589108e-01  1.20000047e-02
  1.99998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  7.75212759e-03 -3.52719043e-01  6.00000000e-01]
Read chunk # 7 out of 10000
[ 0.00000000e+00  7.95731469e+08 -1.03989108e-01  7.00000469e-03
 -8.60001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.65974295e-01 -2.19295880e-01  6.00000000e-01]
Read chunk # 9 out of 10000
torch.Size([111080, 6])
2022-04-07 20:21.43 [debug    ] RoundIterator is selected.
2022-04-07 20:21.43 [info     ] Directory is created at d3rlpy_logs/FQE_2022040720214

Epoch 1/50:   0%|          | 0/878 [00:00<?, ?it/s]

2022-04-07 20:21.45 [info     ] FQE_20220407202143: epoch=1 step=878 epoch=1 metrics={'time_sample_batch': 0.0001402658862242123, 'time_algorithm_update': 0.0020971023954943265, 'loss': 0.0005286302931466156, 'time_step': 0.0023040975144892457, 'init_value': -0.3165595531463623, 'ave_value': -0.3164726957802313, 'soft_opc': nan} step=878


2022-04-07 20:21.45 [info     ] Model parameters are saved to d3rlpy_logs/FQE_20220407202143/model_878.pt


Epoch 2/50:   0%|          | 0/878 [00:00<?, ?it/s]

...

2022-04-07 20:23.25 [info     ] Model parameters are saved to d3rlpy_logs/FQE_20220407202143/model_43022.pt


Epoch 50/50:   0%|          | 0/878 [00:00<?, ?it/s]

2022-04-07 20:23.27 [info     ] FQE_20220407202143: epoch=50 step=43900 epoch=50 metrics={'time_sample_batch': 0.00012191362967523735, 'time_algorithm_update': 0.001705906114165495, 'loss': 0.019594529408642637, 'time_step': 0.00188317087082222, 'init_value': -1.3522834777832031, 'ave_value': -1.3528912111468174, 'soft_opc': nan} step=43900


2022-04-07 20:23.27 [info     ] Model parameters are saved to d3rlpy_logs/FQE_20220407202143/model_43900.pt


[(1,
  {'time_sample_batch': 0.0001402658862242123,
   'time_algorithm_update': 0.0020971023954943265,
   'loss': 0.0005286302931466156,
   'time_step': 0.0023040975144892457,
   'init_value': -0.3165595531463623,
   'ave_value': -0.3164726957802313,
   'soft_opc': nan}),
 (2,
  {'time_sample_batch': 0.00013752896041696326,
   'time_algorithm_update': 0.00205795531392369,
   'loss': 0.0018772853800121275,
   'time_step': 0.002257068499345714,
   'init_value': -0.52862948179245,
   'ave_value': -0.5286661683669388,
   'soft_opc': nan}),
 (3,
  {'time_sample_batch': 0.00014293519673966602,
   'time_algorithm_update': 0.0019880682569429924,
   'loss': 0.003699947557589628,
   'time_step': 0.002199543092679869,
   'init_value': -0.6613953113555908,
   'ave_value': -0.6614020432158646,
   'soft_opc': nan}),
 (4,
  {'time_sample_batch': 0.00014332839731203396,
   'time_algorithm_update': 0.0019716092285642865,
   'loss': 0.00557659785202414,
   'time_step': 0.0021811110012080514,
   'init_va